In [2]:
!unzip /content/drive/MyDrive/Datasets.zip


Streaming output truncated to the last 5000 lines.
  inflating: Datasets/aclImdb/train/unsup/5505_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5506_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5507_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5508_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5509_0.txt  
  inflating: Datasets/aclImdb/train/unsup/550_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5510_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5511_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5512_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5513_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5514_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5515_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5516_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5517_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5518_0.txt  
  inflating: Datasets/aclImdb/train/unsup/5519_0.txt  
  inflating: Datasets/aclImdb/train/unsup/551_0.txt  
  inflating: Dat

In [3]:
import random
import numpy as np
random.seed(1337)
np.random.seed(1337)

In [4]:
import os
import random

dataset_train_pos_path = "Datasets/aclImdb/train/pos/"
dataset_train_neg_path = "Datasets/aclImdb/train/neg/"

dataset_test_pos_path = "Datasets/aclImdb/test/pos/"
dataset_test_neg_path = "Datasets/aclImdb/test/neg/"

In [5]:
def read_dataset(dataset_path, label):
    contents_labels = []
    files = os.listdir(dataset_path)
    for fn in files:
        path = os.path.join(dataset_path, fn)
        with open(path) as f:
            s = f.read()
            contents_labels.append((s, label))
    return contents_labels

In [6]:
train_pos = read_dataset(dataset_train_pos_path, "pos")
train_neg = read_dataset(dataset_train_neg_path, "neg")

test_pos = read_dataset(dataset_test_pos_path, "pos")
test_neg = read_dataset(dataset_test_neg_path, "neg")

In [7]:
train = train_pos + train_neg
test = test_pos + test_neg

In [8]:
random.shuffle(train)
random.shuffle(test)

In [9]:
train_data, y_train = zip(*train)
test_data, y_test = zip(*test)

In [10]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_data)
X_test = vectorizer.transform(test_data)
print("The dimensions of our vectors:")
print(X_train.shape)
print("- - -")

The dimensions of our vectors:
(25000, 74849)
- - -
CPU times: user 14.2 s, sys: 561 ms, total: 14.7 s
Wall time: 21.5 s


In [12]:
%%time

from sklearn.svm import LinearSVC

svm_classifier = LinearSVC()
svm_classifier.fit(X_train, y_train)

predictions = svm_classifier.predict(X_test)

CPU times: user 1.4 s, sys: 98.8 ms, total: 1.5 s
Wall time: 2.95 s


In [13]:
from sklearn.metrics import accuracy_score, classification_report

print("Accuracy: {}\n".format(accuracy_score(y_test, predictions)))
print(classification_report(y_test, predictions))

Accuracy: 0.8772

              precision    recall  f1-score   support

         neg       0.87      0.89      0.88     12500
         pos       0.89      0.87      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



In [16]:
good_review = "The restaurant was really great! I ate wonderful food and had a very good time"
bad_review = "The restuarant was awful. The staff were rude and the food was horrible. I hated it"

restuarant_reviews = [good_review, bad_review]
vectors = vectorizer.transform(restuarant_reviews)
print(svm_classifier.predict(vectors))

['pos' 'neg']
